<center>

# Introduction to Reinforcement Learning

## Getting Home through Q-Learning

</center>

- - -

In [ ]:
%matplotlib notebook

try:
    import numpy as np
except ImportError:
    print("You need to install numpy! Open a command prompt and run 'pip install numpy'")

try:
    import matplotlib.pyplot as plt
except ImportError:
    print("You need to install matplotlib! Open a command prompt and run 'pip install matplotlib'")

- - -

After having secured a master's thesis project at Zenseact, you call your friends to share the news of this unique opportunity. All agree that this needs to be celebrated accordingly, and so you meet up at Andra Långgatan. Your most interesting thesis project is cause for a long and jolly evening, and when you finally decide to go home, it has become quite dark already. To make matters even worse, a strong wind is blowing from south/south-west, and you seem to have forgotten the way to your home in Gamlestaden (for whatever reason - blame it on the long day thinking about falsification of autonomous driving controllers if you'd like). You conclude that your best option would be to start walking in some direction in the hope of finding your home eventually. But careful! If you walk too close alongside the Göta Älv, the strong wind might blow you into the river. In addition to the unpleasant experience, the river will take you back to Järntorget and you need to start your journey home all over again. And if that wasn't bad enough yet, you might get convinced to join some _"late-night studying"_ if you pass by J. A. Pripps at Chalmers.   

In this small assignment, you will implement a Reinforcement Learning algorithm called Q-learning. Reinforcement Learning uses data sampled from the plant (or the environment in RL terms) to derive an optimal controller - just right for finding your way back home.

Let us look at the environment first. As part of the task, you will need to decide how much a visit to J.A. Pripps is worth to you. Note that this is strictly speaking not an element of the Q-learning algorithm, but rather an element of the reward function. While many publications on RL assume that the reward function is given, in practice, designing a good reward function, that enables the RL agent to learn the wanted behavior, is a difficult task. 

In [ ]:
%matplotlib notebook
import random 
from time import sleep

import matplotlib.pyplot as plt

from util import set_up_GBG, plot_pos, plot_Q, plot_average_r, clear_plot


class WindyGothenburg(object):

    def __init__(self, pripps_reward=1.828, test=False):
        """The constructor of the grid world WindyGothenburg"""
        self.w = 12
        self.h = 12
        self.states = {(i, j) for i in range(self.w + 1) for j in range(self.h + 1)}
        self.actions = {'north', 'east', 'south', 'west'}
        
        self._jarntorget = (0, self.h)
        self._home = (self.w, self.h)
        self._gota_alv = {(k, self.h) for k in range(2, self.w)}
        self._chalmers = (3, 2)
        assert 0.01 <= pripps_reward <= 10
        self._pripps_reward = pripps_reward
        self._current_state = self._jarntorget    
        
        # for rendering
        self._test = test
        if not self._test:
            self.fig, self.gw, self.sp = set_up_GBG(self.w, self.h, self._jarntorget, 
                                                    self._chalmers, self._home)
            self.plot_elems = ()
            self.last_pos = None
        
    def _get_next_state(self, state, action):
        """The transition function"""
        i, j = state
        i = i + 1 if random.random() < 0.05 else i # wind blows you to the east
        j = j + 1 if random.random() < 0.1 else j # wind blows you to the north
        if action == 'north':
            j += 1
        elif action == 'east':
            i += 1
        elif action == 'south':
            j -= 1
        elif action == 'west':
            i -= 1
        # ensure the next state is within the grid
        i = max(0, min(i, self.w))
        j = max(0, min(j, self.h))
        return (i, j) # next state

    def step(self, action):
        """
        Advances the simulation by one step
        
        :param action: the control action to apply
        """
        assert action in self.actions
        
        next_state = self._get_next_state(self._current_state, action)
        self._current_state = next_state
        
        if next_state == self._home:
            reward = 100
            done = True
        elif next_state in self._gota_alv:
            reward = -10
            done = True
        elif next_state == self._chalmers:
            reward = random.choice([0, self._pripps_reward])
            done = False
        else:
            reward = 0
            done = False
            
        return next_state, reward, done
    
    def reset(self):
        """Must be called if done == True"""
        self._current_state = self._jarntorget
        return self._current_state
    
    def render(self, Q=None, average_reward=None, episode=None):
        """Renders the environment"""
        assert not self._test, 'Disable test mode for rendering'
        if self.last_pos:
            self.last_pos.remove()
        if self.plot_elems:
            clear_plot(*self.plot_elems)
        self.last_pos = plot_pos(self.gw, self._current_state)
        if Q:
            # Plots the contours of max_u Q(*|x) for each x
            # Plots also the direction of argmax_u Q(*|x) for each x
            self.plot_elems = plot_Q(self.gw, Q, self.w, self.h) 
        if average_reward:
            # Tracking learning over episodes
            assert episode is not None, 'Provide the number of the current episode.'
            plot_average_r(self.sp, average_reward, episode)
        self.fig.canvas.draw()
        sleep(0.001) # So you can enjoy the pretty plot
    
    def close(self):
        if not self._test:
            plt.close(self.fig)

The RL algorithm that you will implement is as follows:

**Algorithm 1.** Q-learning$(\alpha, \epsilon, \gamma)$

>Initialize $Q(x,a)$ arbitrarily
>
>**for all** episodes **do**
>
>>Initialize $x$
>>
>>**for all** steps of episode  **do**
>>>
>>>Choose $a$ in $x$ using policy derived from $Q$ (e.g. $\epsilon$-greedy)
>>>
>>>Take action $u$, observe $r$, $x'$
>>>
>>>$Q(x,a) = Q(x,a) + \alpha \left[r + \gamma \max_{a'} Q(x', a') - Q(x,a) \right]$
>>>
>>>$x = x'$
>>>
>>**end for**
>
>**end for**
>
**return** $\pi(a) = argmax_a Q(x, a)$

We have already implemented parts of this algorithm for you. Make sure to read carefully the functions below. One of those functions creates the Q-table by using nested Python [dictionaries](https://docs.python.org/3/library/stdtypes.html#mapping-types-dict). 

In [ ]:
def initialize_Q(states, actions):
    """
    Initializes the Q-table as a dictionary of dictionaries.
    
    A particular Q-value can be retrieved by calling Q[x][a].
    All actions and their associated values in a state x can 
    be retrieved through Q[x].
    Q-values are initialized to a small random value to encourage 
    exploration and to facilitate learning.
    
    :param states: iterable set of states
    :param actions: iterable set of actions
    """
    return {x: {a: random.random() * 0.1 for a in actions} for x in states}

def argmax_Q(Q, state):
    """Computes the argmax of Q in a particular state."""
    max_q = float("-inf")
    argmax_q = None
    for a, q in Q[state].items():
        if q > max_q:
            max_q = q
            argmax_q = a
    return argmax_q

---

## Task 1

### Finding Home through Q-learning 

As a first task, implement a function that chooses with probability $1-\epsilon$ the action with the highest $Q$-value in a given state (i.e. it chooses greedily), and with probability $\epsilon$ a random action, where $0 < \epsilon < 1$. This is a popular exploration strategy in RL that ensures that all states are visited theoretically infinitively often.

* Implement the $\epsilon$-greedy choice in code. 
* *Hint*: You might want to use the Python function [random.random()](https://docs.python.org/3/library/random.html#https://docs.python.org/3.7/library/random.html#random.random) and [random.choice()](https://docs.python.org/3/library/random.html#random.choice).
* *Hint*: You might want to read the documentation of [dictionaries](https://docs.python.org/3/library/stdtypes.html#mapping-types-dict) again.

In [ ]:
def choose_epsilon_greedily(Q, x, epsilon):
    """
    Chooses random action with probability epsilon, else argmax_a(Q(*|x))
    
    :param Q: Q-table as dict of dicts
    :param x: state
    :param epsilon: float
    """
    # YOUR SOLUTION HERE
    return action

In [ ]:
# space for your own tests

Next, we need to implement a function that decides how fast our RL algorithm will be learning. Generally, the learning rate $\alpha_k$ must satisfy the conditions $\sum_{k=0}^\infty \alpha_k^2 < \infty$ and $\sum_{k=0}^\infty \alpha_k = \infty$, to be able to guarantee that the estimates of Q converge to the optimal Q-function.

* Implement a function that computes $\alpha$ given $k$. 
* *Hint*: Check the lecture notes for further information.

A correct implementation of both functions is needed to complete the task. 

In [ ]:
def get_alpha(k):
    """
    Returns a value of the learning rate.
    :param k: integer index of the update 
    """
    # YOUR SOLUTION HERE
    # E.g. A / (B + k) with some parameters A and B
    return alpha

In [ ]:
# space for your own tests

In [ ]:
# epsilon-greedy tests
Q1 = initialize_Q(states={1}, actions={'a'})
assert choose_epsilon_greedily(Q1, 1, 0.1) == 'a'

Q2 = initialize_Q(states={1}, actions={'a', 'b'})
Q2[1]['a'] = 1
Q2[1]['b'] = 0
assert choose_epsilon_greedily(Q2, 1, 0.0) == 'a'

epsilon = 0.1
k = 0
l = 0
for m in range(1000):
    action = choose_epsilon_greedily(Q2, 1, epsilon)
    k = k + 1 if action == 'a' else k
    l = l + 1 if action == 'b' else l
assert k/m >= (1-epsilon)
assert l/m > 0.0

# learning rate tests
assert 0.0 < get_alpha(1) < 1.0
assert 0.0 < get_alpha(1000) < 1.0
assert 0.0 < get_alpha(1000000) < 1.0
assert 0.0 < get_alpha(1000000000) < 1.0

Finally, we can turn to the $Q$-learning algorithm. We have made a start with it already. 

* Implement the $Q$-value update from Algorithm 1. 

In [ ]:
def learn_q(env, epsilon, gamma, num_episodes=250, max_steps=100, render=False, test=False):
    Q = initialize_Q(env.states, env.actions)
    k = 0
    for l in range(num_episodes):
        # Reset for episode
        x = env.reset()
        done = False
        rewards = 0
        
        for m in range(max_steps):
            # Pick action
            a = choose_epsilon_greedily(Q, x, epsilon)
            next_x, r, done = env.step(a)  
            
            # Update Q-Table
            # YOUR SOLUTION HERE
            
            # Increment
            x = next_x
            rewards += r
            k += 1
            if render:
                env.render(Q)
            if done:
                # Set the Q-values of the terminal state to 0
                for action in Q[next_x].keys():
                    Q[next_x][action] = 0
                break
        
        # Update plots
        if not test:
            env.render(Q, rewards/(m+1), l)
    
    env.close()
    return {x: argmax_Q(Q, x) for x in env.states}


Your task is now to:
* play around with the Q-learning for the WindyGothenburg environment and its parameters. In particular:
 * Choose a value for `pripps_reward`
 * Choose a value for $\epsilon$
 * Choose a value for the discount factor $\gamma$
 * Depending on your implementation of `get_alpha`, their might be also parameters to tune there.
* Your Q-learning will return the learned policy, which is evaluated in the test cell below. 
* If your parameters produces policies that get you home in more than 50% of the cases, we are satisfied.  
* If this increases to more than 75%, we will be happy.  
* _Hint_: If you are ambitious, you will find parameters that have a success rate of 90% and higher. 
* _Hint_: Make sure to understand deeply our evaluation criteria.
* _Hint_: Each parameter affects the learning differently. What values for each parameter would likely produce the wanted behavior? 

In [ ]:
# Choose values for pripps_reward, epsilon, and gamma
pripps_reward = None
epsilon = None
gamma = None

# If you would like to watch the episode, set render = True
# For updates only at the end of each episode, set render = False
# render = False is significantly faster.
render = False

"""
!!! PLEASE COMMENT OUT THE TWO LINES BELOW AGAIN BEFORE SUBMISSION !!!
"""

# env = WindyGothenburg(pripps_reward)
# control_policy = learn_q(env, epsilon, gamma, render=render)

"""
!!! PLEASE COMMENT OUT THE TWO LINES ABOVE AGAIN BEFORE SUBMISSION !!!
"""

In [ ]:
def policy_success_rate(pripps_reward, epsilon, gamma):
    solved = 0
    for _ in range(200):    
        env = WindyGothenburg(pripps_reward, test=True)
        control_policy = learn_q(env, epsilon=epsilon, gamma=gamma, num_episodes=250,
                                 max_steps=100, render=False, test=True)
        x = env.reset()
        done = False
        rewards = []
        i = 0
        while not done and i < 20:
            x, r, done = env.step(control_policy.get(x))
            rewards.append(r)
            i += 1
        solved += 1 if 100 in rewards else 0
    return solved/200

rate = policy_success_rate(pripps_reward, epsilon, gamma)
assert rate > 0.50, 'Got {} instead'.format(rate)

In [ ]:
assert rate > 0.75, 'Got {} instead'.format(rate)
print("The achieved success rate was ", rate)

---

## Task 2

### Reflections on Q-learning 

* Reflect on this part of the assignment and write down your insights in a few brief sentences. 
* Make sure to touch upon:
 * the final value of `pripps_reward` that you chose and why you have chosen that value;
 * the impact of `pripps_reward` on solving the problem
 * the final value of $\epsilon$ that you chose and why you have chosen that value;
 * the final value of $\gamma$ that you chose and why you have chosen that value;
 * the final value/range of $\alpha$ that you chose and why you have chosen that/those value/s;
 * the process of finding those parameters;
 * the contour of the Q-function over time;
 * and whether you think Q-learning would be a good idea to try next time you are lost at Järntorget. 

---

That is all there is. If you are done,

* Save the notebook
* Upload the .ipynb file to Canvas
* Tell your teammate how much you appreciated their invaluable insights and how fun it was to collaborate with them on the assignments.